In [2]:
from PIL import Image
import pandas as pd
from keras.models import load_model
import numpy as np
from numpy import asarray
from numpy import expand_dims
import requests
from io import BytesIO
import pickle
import cv2
from keras_facenet import FaceNet

In [3]:
#HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
#MyFaceNet = load_model('facenet_keras.h5')
MyFaceNet = FaceNet()

In [4]:
myfile = open("data2.pkl", "rb")
database = pickle.load(myfile)
myfile.close()
url ="http://192.168.1.16/cam-lo.jpg"

In [5]:
# Đường dẫn đến file CSV
csv_path = "attendance.csv"

# Tải file CSV vào DataFrame
attendance_df = pd.read_csv(csv_path)

# Ngưỡng để xác định khớp
threshold = 0.6  # Điều chỉnh giá trị này tùy theo độ chính xác của mô hình

while True:
    # Lấy ảnh từ ESP32-CAM
    response = requests.get(url)
    img_data = response.content
    img_array = np.array(bytearray(img_data), dtype=np.uint8)
    gbr1 = cv2.imdecode(img_array, -1)

    # Kiểm tra xem ảnh có bị lỗi không
    if gbr1 is None:
        print("Không thể lấy ảnh từ ESP32-CAM!")
        break

    # Phát hiện khuôn mặt trong ảnh
    wajah = HaarCascade.detectMultiScale(gbr1, 1.1, 4)

    if len(wajah) > 0:
        # Nếu phát hiện khuôn mặt
        x1, y1, width, height = wajah[0]
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height

        # Chuyển ảnh từ OpenCV (BGR) sang RGB
        gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
        gbr = Image.fromarray(gbr)  # Chuyển từ OpenCV sang PIL
        gbr_array = asarray(gbr)

        # Cắt khuôn mặt trong ảnh
        face = gbr_array[y1:y2, x1:x2]
        face = Image.fromarray(face)
        face = face.resize((160, 160))  # Đảm bảo kích thước face chuẩn
        face = asarray(face)
        face = expand_dims(face, axis=0)
        
        # Lấy đặc trưng khuôn mặt từ mô hình (ví dụ: FaceNet)
        signature = MyFaceNet.embeddings(face)

        # Tìm người có đặc trưng gần nhất trong cơ sở dữ liệu
        min_dist = 10
        identity = 'Unknown'  # Mặc định là không nhận diện được
        for key, signatures in database.items():
            for stored_signature in signatures:
                dist = np.linalg.norm(stored_signature - signature)
                if dist < min_dist:
                    min_dist = dist
                    identity = key

        # Kiểm tra nếu nhận diện được khuôn mặt (khoảng cách nhỏ hơn threshold)
        if min_dist < threshold:
            # Nhận diện thành công -> chữ và khung màu xanh
            cv2.putText(gbr1, identity, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Cập nhật trạng thái trong file CSV
            if identity in attendance_df['Name'].values:
                # Lấy chỉ số của người trong file CSV
                index = attendance_df[attendance_df['Name'] == identity].index[0]
                
                # Kiểm tra trạng thái hiện tại
                if attendance_df.at[index, 'How'] == "absent":
                    attendance_df.at[index, 'How'] = "present"
                    print(f"Cập nhật trạng thái cho {identity}: present")
                    # Ghi lại các thay đổi vào file CSV
                attendance_df.to_csv(csv_path, index=False)
                print("Cập nhật file CSV thành công!")
        else:
            # Không nhận diện được -> chữ màu đỏ
            cv2.putText(gbr1, "Unknown", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 0, 255), 2)
    else:
        # Nếu không phát hiện được khuôn mặt -> Hiển thị thông báo
        cv2.putText(gbr1, "No face detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    # Ghi lại các thay đổi vào file CSV
    attendance_df.to_csv(csv_path, index=False)
    print("Cập nhật file CSV thành công!")
    # Hiển thị kết quả
    cv2.imshow('res', gbr1)

    # Nhấn ESC để thoát
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # ESC
        break

# Ghi lại các thay đổi vào file CSV
attendance_df.to_csv(csv_path, index=False)
print("Cập nhật file CSV thành công!")

cv2.destroyAllWindows()

Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Cập nhật file CSV thành công!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Cập nhật file